In [1]:
import keras
import pandas as pd
import numpy as np
from keras.layers import *
from keras.activations import *
from keras.callbacks import *
from keras.optimizers import *
from keras.models import *
import matplotlib.pyplot as plt
from keras.utils import to_categorical
from sklearn.preprocessing import MinMaxScaler
from scipy import signal
import stldecompose

Using TensorFlow backend.


In [2]:
data_orig = pd.read_csv('../data/files/multiple_concatenated_tickers.csv')
data_orig.tail()

,Index,QQQ.Open,QQQ.High,QQQ.Low,QQQ.Close,QQQ.Volume,QQQ.Adjusted,TSLA.Open,TSLA.High,TSLA.Low,...,TLRY.Low,TLRY.Close,TLRY.Volume,TLRY.Adjusted,MU.Open,MU.High,MU.Low,MU.Close,MU.Volume,MU.Adjusted
863,2019-06-10,182.250000,184.850006,182.210007,183.149994,41385300,183.149994,210.250000,216.940002,209.009995,...,41.029999,43.139999,7322600,43.139999,34.599998,35.540001,34.520000,34.939999,21728000,34.939999
864,2019-06-11,185.059998,185.399994,182.779999,183.399994,41260300,183.399994,219.139999,220.899994,213.500000,...,40.139999,40.490002,2898900,40.490002,35.799999,35.990002,34.750000,34.840000,19208600,34.840000
865,2019-06-12,182.899994,183.279999,182.000000,182.339996,27758100,182.339996,222.949997,223.380005,209.000000,...,39.430000,41.820000,1702500,41.820000,34.009998,34.099998,32.730000,32.959999,28746500,32.959999
866,2019-06-13,183.100006,183.869995,182.740005,183.419998,23715800,183.419998,210.380005,214.899994,207.509995,...,40.400002,40.700001,1167200,40.700001,33.080002,33.639999,33.009998,33.380001,16586700,33.380001
867,2019-06-14,182.479996,183.110001,181.940002,182.639999,22834000,182.639999,211.250000,216.649994,210.399994,...,38.700001,39.009998,1368200,39.009998,32.450001,32.840000,32.240002,32.660000,19700800,32.660000


In [4]:
vol_data_orig = pd.read_csv('../data/files/multiple_concatenated_tickers_volatility.csv')
# vol_data20_orig = pd.read_csv('../data/files/multiple_concatenated_tickers_volatility20.csv')
#vol_data50_orig = pd.read_csv('../data/files/multiple_concatenated_tickers_volatility50.csv')
#vol_data100_orig = pd.read_csv('../data/files/multiple_concatenated_tickers_volatility100.csv')
vol_data_orig.tail()

,Index,volatilityQQQ,volatilityTSLA,volatilityMSFT,volatilityINTC,volatilityAAPL,volatilityNFLX,volatilityAMZN,volatilityFB,volatilityGOOG,...,volatilityXLY,volatilityXLP,volatilityXLV,volatilityXLF,volatilityXLK,volatilityXTL,volatilityXLU,volatilityXLRE,volatilityTLRY,volatilityMU
863,2019-06-10,0.262443,0.574566,0.333510,0.253769,0.281142,0.381002,0.409732,0.517354,0.406953,...,0.202950,0.150711,0.130785,0.183441,0.267121,0.202553,0.168077,0.155938,1.063379,0.412362
864,2019-06-11,0.253383,0.573597,0.326130,0.252188,0.269323,0.369345,0.404458,0.531756,0.402374,...,0.187672,0.134942,0.109451,0.181765,0.260561,0.200543,0.150839,0.130113,1.092160,0.414424
865,2019-06-12,0.259465,0.628408,0.331261,0.266931,0.278214,0.377129,0.405765,0.535325,0.401467,...,0.189846,0.136326,0.109338,0.192895,0.268465,0.203165,0.158621,0.129671,1.072741,0.527995
866,2019-06-13,0.231038,0.605956,0.304988,0.239902,0.232847,0.353386,0.375983,0.518845,0.404909,...,0.156865,0.076984,0.091795,0.159182,0.234898,0.125690,0.158864,0.127678,1.070979,0.517080
867,2019-06-14,0.172617,0.533444,0.202580,0.234044,0.224527,0.340522,0.205248,0.225017,0.167734,...,0.131051,0.074474,0.099594,0.159437,0.207356,0.149072,0.159537,0.127758,0.946152,0.533473


In [5]:
data_orig = data_orig\
  .merge(vol_data_orig, how="inner", left_on=data_orig.Index, right_on=vol_data_orig.Index).fillna(method="ffill")\
  .drop(["key_0", "Index_y"], axis=1)

In [6]:
[i for i in list(data_orig.columns) if i == 'Index']

[]

In [17]:
import pickle
import os
from keras.preprocessing import *
from nltk.stem import SnowballStemmer
from dateutil.parser import parse
from datetime import datetime

data = []
for i in os.listdir("../briefings/"):
    #print(i)
    f = open("../briefings/"+str(i), "rb")
    d = pickle.loads(f.read())
    data.append(d)

def categories(n, p):
    n = float(n)
    p = float(p)
    if n >= p:
        return np.array([0, 0, 1])
    if n <= -p:
        return np.array([1, 0, 0])
    else:
        return np.array([0, 1, 0])
    
#Headlines
headlines = []
text_bodies = []
all_text = []
dates = []
targets = []
snow = SnowballStemmer('english')
for i,v in enumerate(data):
    for k in v:
        if k["date"] == -1:
            continue
        else:
            ticker_data = k["tickerinfo"]
            if not ticker_data:
                continue
            if len(ticker_data[1]) == 4:
                w = parse(k["date"])
                targets.append([w, categories(ticker_data[1][3].replace("(","").replace(")","").replace("%",""), 1.0)]) # for regression
                headlines.append(''.join([snow.stem(w) for w in str(k["headline"])]))
                text_bodies.append(''.join([snow.stem(w) for w in str(k["text"])]))
                all_text.append(''.join([snow.stem(w) for w in str(k["headline"])]+[snow.stem(w) for w in str(k["text"])]))
                dates.append(w.date().isoformat())
#                 all_text.append(''.join([snow.stem(w) for w in str(k["text"])]))
                dates.append(w.date().isoformat())
            else:
                continue

tokenizer = text.Tokenizer(num_words=500, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower=True, split=' ', char_level=False, oov_token=None)

In [18]:
targets_df = pd.DataFrame(targets)
targets_df["DATE"] = [i[0].isoformat(sep=' ').split()[0] for i in targets]
# targets_df = targets_df.iloc[:,1:].groupby(['DATE'], as_index=True)
# targets_df.columns = ["pct_change_cat"]
targets_df.columns = ["full_ts", "pct_change_cat", "DATE"]
targets_df.iloc[:,1]

0        [0, 1, 0]
1        [0, 1, 0]
2        [0, 1, 0]
3        [0, 1, 0]
4        [0, 1, 0]
5        [0, 1, 0]
6        [0, 1, 0]
7        [0, 1, 0]
8        [0, 1, 0]
9        [0, 1, 0]
10       [0, 1, 0]
11       [0, 1, 0]
12       [0, 1, 0]
13       [0, 1, 0]
14       [1, 0, 0]
15       [0, 1, 0]
16       [0, 1, 0]
17       [0, 1, 0]
18       [0, 1, 0]
19       [0, 1, 0]
20       [0, 1, 0]
21       [0, 1, 0]
22       [0, 1, 0]
23       [0, 1, 0]
24       [0, 1, 0]
25       [0, 1, 0]
26       [0, 1, 0]
27       [0, 1, 0]
28       [0, 1, 0]
29       [0, 1, 0]
           ...    
12103    [0, 1, 0]
12104    [0, 1, 0]
12105    [0, 1, 0]
12106    [0, 1, 0]
12107    [0, 1, 0]
12108    [0, 1, 0]
12109    [0, 1, 0]
12110    [0, 1, 0]
12111    [0, 1, 0]
12112    [0, 1, 0]
12113    [0, 1, 0]
12114    [0, 1, 0]
12115    [0, 1, 0]
12116    [0, 1, 0]
12117    [0, 1, 0]
12118    [0, 1, 0]
12119    [0, 1, 0]
12120    [0, 1, 0]
12121    [0, 1, 0]
12122    [0, 1, 0]
12123    [0, 1, 0]
12124    [0,

In [19]:
tokenizer.fit_on_texts(all_text)
encoded_docs = tokenizer.texts_to_matrix(all_text, mode='tfidf')
tfidf_df = pd.DataFrame(encoded_docs)
# tfidf_df["DATE"] = dates
# tfidf_docs = tfidf_df.groupby(['DATE'], as_index=True).mean()
# tfidf_df

In [20]:
X = tfidf_df.as_matrix()
# y = targets_df.as_matrix() # predict current
y = targets_df.iloc[:,1].shift(-1).dropna().as_matrix().reshape(-1,1) # predict next day

X = np.array([[i] for i in X]) # for batch size 1
X = X[1:]
y = np.array([i[0] for i in y])

print(X.shape)
print(y.shape)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

print("x_train", x_train.shape)
print("y_train", y_train.shape)
print("x_test", x_test.shape)
print("y_test", y_test.shape)

(12132, 1, 500)
(12132, 3)
x_train (9705, 1, 500)
y_train (9705, 3)
x_test (2427, 1, 500)
y_test (2427, 3)


/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
LAYERS = 300
model = Sequential()

# Original Architecture
# model.add(Dense(
#     input_dim=x_train.shape[1],
#     output_dim=LAYERS))
# model.add(Dropout(0.2))
# model.add(Dense(
#     input_dim=x_train.shape[1],
#     output_dim=LAYERS))
# #model.add(Dense(output_dim = y_train.shape[1]))
# model.add(Dense(output_dim=y_train.shape[1]))
# model.add(Activation('linear'))

# Stateful LSTM Architecture
model.add(LSTM(
    batch_input_shape=(1, 1, 500),
    output_dim=LAYERS,
    return_sequences=True,
    stateful=True))
model.add(BatchNormalization())
model.add(LSTM(
    LAYERS,
    return_sequences=False,
    stateful=True))
model.add(Dropout(0.5))
# model.add(Dense(output_dim=y_train.shape[1]))
model.add(Dense(3))
model.add(Activation('softmax'))

start = time.time()
adam = keras.optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=adam)
print('compilation time : ', time.time() - start)

/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(batch_input_shape=(1, 1, 500..., return_sequences=True, stateful=True, units=300)`


compilation time :  0.0242159366607666


In [22]:
MODELNAME = 'briefing_nextday_stateful_lstm_softmax'
earlyStopping = EarlyStopping(monitor='val_loss', patience=6, verbose=0, mode='min')
mcp_save = ModelCheckpoint('./models/'+MODELNAME+'_best.hdf5', save_best_only=True, monitor='val_loss', mode='min')
reduce_lr_loss = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, verbose=1, epsilon=1e-4, mode='min')
tbrd = TensorBoard(log_dir="./logs", histogram_freq=0, batch_size=128, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None, embeddings_data=None, update_freq='epoch')

max_len = 20
class ResetStatesCallback(Callback):
    def __init__(self):
        self.counter = 0

    def on_batch_begin(self, batch, logs={}):
        if self.counter % max_len == 0:
            self.model.reset_states()
        self.counter += 1

VALIDATIONSIZE = 0.10
EPOCHS = 5000
# model = keras.models.load_model('./models/'+MODELNAME+'_best.hdf5') 
history = model.fit(
    x_train,
    y_train,
    batch_size=1,
    nb_epoch=EPOCHS,
    validation_split=VALIDATIONSIZE,
    callbacks = [reduce_lr_loss, earlyStopping, mcp_save, tbrd, ResetStatesCallback()],
    shuffle=False)

/usr/local/anaconda3/lib/python3.6/site-packages/keras/callbacks.py:1065: UserWarning: `epsilon` argument is deprecated and will be removed, use `min_delta` instead.
  warnings.warn('`epsilon` argument is deprecated and '
/usr/local/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Train on 8734 samples, validate on 971 samples
Epoch 1/5000
8734/8734 [==============================] - 235s 27ms/step - loss: 2.9105 - val_loss: 2.8855
Epoch 2/5000
6996/8734 [=======================>......] - ETA: 48s - loss: 2.9260

KeyboardInterrupt: 

In [161]:
# Write callback that looks for loss below threshold and that loss and val_loss are less than 0.01 difference

In [228]:
# model.evaluate(x_test, y_test)
model.evaluate(x_test, y_test, batch_size=1)

172/172 [==============================] - 2s 10ms/step


0.9353546145132183

In [229]:
model = keras.models.load_model('./models/'+MODELNAME+'_best.hdf5') 
# model.evaluate(x_test, y_test)
model.evaluate(x_test, y_test, batch_size=1)

172/172 [==============================] - 3s 16ms/step


0.884816817182455

In [234]:
# print(model.predict(tfidf_docs.loc["2016-01-06"].as_matrix().reshape(1,-1))) # known down day
# print(model.predict(tfidf_docs.loc["2016-03-09"].as_matrix().reshape(1,-1))) # known up day
print(model.predict(np.array([tfidf_docs.loc["2016-01-06"].as_matrix().reshape(1,-1)])))
print(model.predict(np.array([tfidf_docs.loc["2016-03-09"].as_matrix().reshape(1,-1)])))

def predict_based_on_text(text):
#     return model.predict(tokenizer.texts_to_matrix([text]).reshape(1,-1))
    return model.predict(np.array([tokenizer.texts_to_matrix([text]).reshape(1,-1)]))

print(" ")
print(predict_based_on_text("U.S. dollar index weakens"))
print(predict_based_on_text("trade talks begin"))
print(predict_based_on_text("crude oil miss"))

# 6/14
print(" ")
print(predict_based_on_text("Large Cap Underperformance Weighs On Dow"))
print(predict_based_on_text("Wall Street slips on semiconductor weakness"))
print(predict_based_on_text("FAANG stocks trade mixed"))
print(predict_based_on_text("Amazon underpinning resiliency in stock market this week"))
print(predict_based_on_text("Facebook gains on crypto plans"))

[[0.14092773]]
[[0.14935726]]
 
[[0.10491294]]
[[0.08785099]]
[[0.08546847]]
 
[[0.0851447]]
[[0.16033787]]
[[0.6232299]]
[[0.2937979]]
[[0.16033787]]


/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/root/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
